# Sepic Converter [3 V, 20 V] -> 5.5 V
This is the SEPIC converter meant for converting the supply board voltage (3, 20) V into a 5.5 V board supply voltage. (Taking into account the diode drop)

## Inductor sizing
We can choose 2 inductors wound around the same common core. This would 
- Give us more boardspace
- Require smaller inductors in terms of magnetization (half as big)

This option seems to be more expensive however, so think about simply buying twice the inductor, since the board-size is not a limitation anyways.

### Variables required for calculation

In [2]:

#############! DESIGN PARAMETERS ##################

### Take from LTSpice simulation + add on top
I_rms_A = 0.5 # Amps (slightly overdimensioned already)
I_peak_A = 1 # Amps, for each bridge spike of 1 amp 180 degrees shifted

### DIODE Choice:
# 40 V max
# 1 A RMS current max
V_diode = 0.55 # SS14
V_max_diode = 20 # 

### MOSFET Choice: 
# AO3400A-mosfet -> To low Vds
# NCE6005AS-mosfet -> usable in full bridge as well as sepic + MUCH cheaper
Vfet_gsth = 1.6 # Volt
Vfet_diode = 1.2 # Volt
Rfet_dsn = 32e-3 # At 3 amps, 2.5 V
Vfet_dsn = Rfet_dsn * I_rms_A
Vds_max = 60 # V
Vgs_max = 20 # V
Id_max = 5 # A
Id_peak = 24 # A (pulsed drain current)
print(f"Vfet_dsn {Vfet_dsn}")

# Required Output Voltage
Vout = 5.5

Vfet_dsn 0.016


### Calculations for (Vin_min = 5, Vin_max = 20V)

In [3]:

#! >>> SET INPUT VOLTAGE min max
Vin_list = [20, 3.3]
#! <<< SET INPUT VOLTAGE

In [4]:
for Vin in Vin_list:
	print(f"\r\n##################################################")
	print(f"################# VIN == {Vin} ###################")
	print(f"##################################################")

	D_sepic = (Vout + V_diode) / (Vout + Vin - Rfet_dsn * I_rms_A  + V_diode)
	print(f"Duty cycle: {D_sepic}, time the switch is closed")

	if (D_sepic > 0.5):
		print(f"Warning slope compensation required")

	##### MINIMUM INDUCTOR SIZE #######
	print(f"\r\nMINIMUM INDUCTOR SIZE")
	freq_sw = 500e3

	I1_sepic = D_sepic * I_rms_A / (1-D_sepic)
	I2_sepic = I_rms_A

	# Inductance
	L1_sepic = (Vin - Vfet_dsn) * (1-D_sepic) / (2 * I_rms_A * freq_sw)
	L2_sepic = (Vin - Vfet_dsn) * (D_sepic) / (2 * I_rms_A * freq_sw)

	print(f"L1_sepic: {L1_sepic*1e6} uH, L2_sepic: {L2_sepic*1e6} uH")


##################################################
################# VIN == 20 ###################
##################################################
Duty cycle: 0.23238841514941996, time the switch is closed

MINIMUM INDUCTOR SIZE
L1_sepic: 30.679899823307984 uH, L2_sepic: 9.288100176692017 uH

##################################################
################# VIN == 3.3 ###################
##################################################
Duty cycle: 0.648167988000857, time the switch is closed
Warning slope compensation required

MINIMUM INDUCTOR SIZE
L1_sepic: 2.3108326548103713 uH, L2_sepic: 4.257167345189628 uH


### SEPIC passives sizing
(check [sepic_fetdriver.ipynb](sepic_fetdriver.ipynb) for more details)

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#! >>> INSERT CHOICE
# Set ESR of output capacitor
ESR_cout = 1e-3 # ESR of output capacitor
L1_ch = 60e-6#L1_sepic
L2_ch = 60e-6#L2_sepic
#! <<< INSERT CHOICE
import math 
for Vin in Vin_list:
	print(f"\r\n##################################################")
	print(f"################### VIN == {Vin} #####################")
	print(f"##################################################")

	D_sepic = (Vout + V_diode) / (Vout + Vin - Rfet_dsn * I_rms_A  + V_diode)
	print(f"Duty cycle: {D_sepic}, time the switch is closed")

	if (D_sepic > 0.5):
		print(f"Warning slope compensation required")

	##### MINIMUM INDUCTOR SIZE #######
	print(f"\r\nMINIMUM INDUCTOR SIZE")
	freq_sw = 500e3

	I1_sepic = D_sepic * I_rms_A / (1-D_sepic)
	I2_sepic = I_rms_A

	# Inductance
	L1_sepic = (Vin - Vfet_dsn) * (1-D_sepic) / (2 * I_rms_A * freq_sw)
	L2_sepic = (Vin - Vfet_dsn) * (D_sepic) / (2 * I_rms_A * freq_sw)

	print(f"L1_sepic: {L1_sepic*1e6} uH, L2_sepic: {L2_sepic*1e6} uH")
	
	print(f"\r\nINDUCTOR CHOICE")
	print(f"L1_ch: {L1_ch*1e6} uH, L2_ch: {L2_ch*1e6} uH")

	##### PEAK CURRENT CALCULATION #######
	print(f"\r\n#### PEAK CURRENT CALCULATION ####")
	# Peak current
	dIl1 = (Vin - Vfet_dsn) * D_sepic / (L1_ch * freq_sw)
	dIl2 = (Vin - Vfet_dsn) * D_sepic / (L2_ch * freq_sw)
	print(f"dIl1: {dIl1} A, dIl2: {dIl2} A")

	Il1_peak = I1_sepic + dIl1 / 2
	Il2_peak = I2_sepic + dIl2 / 2
	print(f"Il1_peak: {Il1_peak} A, Il2_peak: {Il2_peak} A")

	######## OUTPUT VOLTAGE RIPPLE #########
	print(f"\r\n#### OUTPUT VOLTAGE RIPPLE (ESR-DEPENDENT) ####")

	dVout = ((I_rms_A / (1-D_sepic)) + dIl2 / 2) * ESR_cout
	print(f"dVout: {dVout} V")

	######## SEPIC CAPACITOR SELECTION #########
	print(f"\r\n#### SEPIC CAPACITOR SELECTION ####")
	I_sw_peak = I1_sepic + I_rms_A + (dIl1 + dIl2) / 2
	I_swrms = math.sqrt((I_sw_peak**2 - I_sw_peak*(dIl1 + dIl2) + (dIl1 + dIl2)**2 / 3)*D_sepic)
	print(f"I_SW_RMS: {I_swrms} A")
	print(f"I_SW_PEAK: {I_sw_peak} A")

	I_cap_rms = math.sqrt(I_swrms**2 + (Il1_peak**2-Il1_peak*dIl1+dIl1**2) * (1-D_sepic))
	V_cap_rms = Vin
	C_min = L1_ch * I_rms_A**2 / (Vin-Vfet_dsn)**2

	print(f"[C_coupling_minimum] {C_min*1e6} uF, I_cap_rms {I_cap_rms} A, V_max: {V_cap_rms} V")

	######## INPUT CAPACITOR SELECTION #########
	print(f"\r\n#### INPUT CAPACITOR SELECTION ####")
	I_cin_rms = (D_sepic / (2 * math.sqrt(3))) * ((Vin - Vfet_dsn) / (L1_ch * freq_sw))
	print(f"Input capacitor RMS current {I_cin_rms} A")


	######## OUTPUT CAPACITOR SELECTION #########
	print(f"\r\n#### OUTPUT CAPACITOR SELECTION ####")
	I_cout_rms = math.sqrt((I_sw_peak**2 - I_sw_peak * (dIl1 + dIl2) + ((dIl2+ dIl1)**2)/3.0) * (1-D_sepic) - I_rms_A**2)
	print(f"output capacitor RMS current: {I_cout_rms} A")


	######## SLOPE COMPENSATION #######
	print(f"\r\n#### SLOPE COMPENSATION | SENSE RESISTOR SELECTION ####")
	V_sense = 0.156 # Volt - FIXED current sense threshold voltage
	V_sl = 92e-3 # Internal compensation ramp voltage
	V_sl_ratio = 0.49
	R_sl = 1
	K_sl = 40e-6
	dV_sl = K_sl * R_sl

	### R_sen estimation (boost equations)
	R_sense = (V_sense - D_sepic * (V_sl + dV_sl)) / I_sw_peak # (eq. 48)
	print(f"R_sense: {R_sense} Ohm")
	print(f"I_max: {V_sense / R_sense} A")
	### Slope compensation
	# R_sl = ((R_sense * (Vout - 2*Vin)) / (2*freq_sw*L1_ch) - V_sl) / K_sl
	print(f"Slope compensation resistor: {R_sl}, Duty: {D_sepic}")


##################################################
################### VIN == 20 #####################
##################################################
Duty cycle: 0.23238841514941996, time the switch is closed

MINIMUM INDUCTOR SIZE
L1_sepic: 30.679899823307984 uH, L2_sepic: 9.288100176692017 uH

INDUCTOR CHOICE
L1_ch: 60.0 uH, L2_ch: 60.0 uH

#### PEAK CURRENT CALCULATION ####
dIl1: 0.15480166961153363 A, dIl2: 0.15480166961153363 A
Il1_peak: 0.22877193168326881 A, Il2_peak: 0.5774008348057669 A

#### OUTPUT VOLTAGE RIPPLE (ESR-DEPENDENT) ####
dVout: 0.0007287719316832689 V

#### SEPIC CAPACITOR SELECTION ####
I_SW_RMS: 0.3169464335597079 A
I_SW_PEAK: 0.8061727664890356 A
[C_coupling_minimum] 0.03756007207687687 uF, I_cap_rms 0.3630971141774226 A, V_max: 20 V

#### INPUT CAPACITOR SELECTION ####
Input capacitor RMS current 0.04468739281061122 A

#### OUTPUT CAPACITOR SELECTION ####
output capacitor RMS current: 0.2860369144114464 A

#### SLOPE COMPENSATION | SENSE RESISTOR SELECTI

## Setting output voltage
- Don't forget to connect an output capacitor between feedback and ground pins
	- Reduces bandwidth
	- Reduces susceptibility to noise

In [9]:
import numpy as np

RF1 = 60e3 # 60 kOhms feedback in example
RF2 = (1.26 * RF1) / (Vout - 1.26)

print(f"Resistors: {RF1*1e-3:.2f} kOhm, {RF2*1e-3:.2f} kOhm")


# (RF2 / (RF1+RF2)) * Vout = 1.26 V -> this is the equation we have to rework to RF2 / RF1
# Vout / 1.26 = 1 + RF1 / RF2 
# Vout / 1.26 - 1 = RF1 / RF2
# RF2 = (1.26 V x RF1) / (Vout - 1.26 V)

def get_resistor_ratio(v_out):
	return ((v_out / 1.26) - 1)

resistor_array = np.array([0.470, 0.51, 0.56, 0.68, 0.82, 1, 1.2, 1.5, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.6, 3.9, 4.7, 4.99, 5.1, 5.6, 6.2, 
						6.8, 7.5, 8.2, 9.1, 10, 12, 13, 15, 20, 22, 24, 27, 30, 33, 39, 47, 51, 56, 68, 75, 82, 100, 150])
v_target = 5.5
RF1 = 9.1e3
RF2 = 2.7e3

######### DDR3L
print(f"---- V = {v_target} ----")
v_target_ratio = get_resistor_ratio(v_target)
r2_target_arr = np.round(resistor_array / v_target_ratio, 2)
r1_target_arr = np.round(resistor_array * v_target_ratio, 2)
print(f"{v_target} r1/r2: {v_target_ratio}")
print(f"R2 values {r2_target_arr}")
print(f"R1 values {r1_target_arr}")

for r1, r2 in zip(r1_target_arr, r2_target_arr):
	if (r1 in resistor_array):
		print(f"Perfect r1: {r1}, r2: {r1 / v_target_ratio}")
	if (r2 in resistor_array):
		print(f"Perfect r2 found: {r2}, r1: {r2 * v_target_ratio}")

vout_actual =1.26*(1+RF1/RF2)
print(f"offset: {((vout_actual - v_target) / v_target) * 100:.2f}%, vout: {vout_actual}")

I_fb = vout_actual / (RF1+RF2)
print(f"Power out: {vout_actual*I_fb}")

Resistors: 60.00 kOhm, 17.83 kOhm
---- V = 5.5 ----
5.5 r1/r2: 3.3650793650793647
R2 values [ 0.14  0.15  0.17  0.2   0.24  0.3   0.36  0.45  0.53  0.59  0.65  0.71
  0.8   0.89  0.98  1.07  1.16  1.4   1.48  1.52  1.66  1.84  2.02  2.23
  2.44  2.7   2.97  3.57  3.86  4.46  5.94  6.54  7.13  8.02  8.92  9.81
 11.59 13.97 15.16 16.64 20.21 22.29 24.37 29.72 44.58]
R1 values [  1.58   1.72   1.88   2.29   2.76   3.37   4.04   5.05   6.06   6.73
   7.4    8.08   9.09  10.1   11.1   12.11  13.12  15.82  16.79  17.16
  18.84  20.86  22.88  25.24  27.59  30.62  33.65  40.38  43.75  50.48
  67.3   74.03  80.76  90.86 100.95 111.05 131.24 158.16 171.62 188.44
 228.83 252.38 275.94 336.51 504.76]
Perfect r2 found: 2.7, r1: 9.085714285714285
offset: 0.12%, vout: 5.506666666666667
Power out: 0.002569777777777778


## Setting Current Limit

In [26]:

#! >>> INSERT COUPLING CAPACITOR CHOICE HERE
C_coupling_ch = 1e-6 # F
D_sepic_max = 0.65
Vin = 5
Cin_ch = 100e-6
Cout_ch = 100e-6
#! <<< INSERT COUPLING CAPACITOR CHOICE HERE

# LOAD POLE ESTIMATION
print(f"\r\n##################################################")
print(f"################### VIN == {Vin} #####################")
print(f"##################################################")
# RIGHT HALF PLANE ZERO
f_RHPZ = (1 - D_sepic_max) ** 2 * Vout / (2 * math.pi * D_sepic_max * L2_ch * 0.5 * I_rms_A)
print(f"Right Half plane zero: {f_RHPZ*1e-3:.2f}kHz")
# RESONANT LC POLE 
f_R = 1 / (2*math.pi * math.sqrt(L2_ch * C_coupling_ch))
print(f"Cs-L2 Resonance: {f_R*1e-3:.2f}kHz")
f_crossover = min(f_R, f_RHPZ) / 6
print(f"Crossover frequency: {f_crossover*1e-3:.2f}kHz")

#### COMPENSATION NETWORK CHOICE
# CHOICE OF RC
Gcs = 1/R_sense # Current sense gain
print(f"Gcs: {Gcs}")
Gma = 800e-6 # Error Amplifier Transconductance
Rc = (2 * math.pi * f_crossover * Cout_ch * Vout**2 * (1+D_sepic_max))
Rc = Rc / (Gcs*Gma*1.26*Vin*D_sepic_max)
print(f"Rc: {Rc*1e-3:.2f}kOhm")

# CHOICE OF CC1 (1/4th crossover frequency)
Cc1 = 4 / (2 * math.pi * f_crossover * Rc)
Cc2 = Cout_ch * ESR_cout / Rc
print(f"Cc1: {Cc1*1e9}nF, Cc2: {Cc2*1e9}nF")


##################################################
################### VIN == 5 #####################
##################################################
Right Half plane zero: 11.00kHz
Cs-L2 Resonance: 20.55kHz
Crossover frequency: 1.83kHz
Gcs: 15.487284641299574
Rc: 1.13kOhm
Cc1: 306.5377866944416nF, Cc2: 8.826061272879485nF


## Mosfet / Diode Selection

#### Using the AO3400A
It seems to be better to select a MOSFET like AO3400A, since it has a much lower Vgs(th) and thus has a much larger range at which it can supply power to the processing-board.
- Input voltage will be max 20 V
- Output voltage will be max 9-10 V
- Vgs will be max 7 V (limit is 12 V)

**Stats:**
- Vgs(th): 1-2 V
- Id_max: 4-5 A
- Pd: 1.4 W
- Vgs: 12 V max
- Vds_max: 30 V
- Rdson: 30 mOhm

#### Finding an LTSpice Model
Use the IRLML6344PbF FET model for the SEPIC instead of the AO3400A.

**Stats:**
- Vgs(th): 0.5-1.1 V
- Id_max: 7 A
- Pd: 1.2 W
- Vgs: 12 V max
- Vds_max: 20 V
- Rds_on: 30 mOhm

In [228]:
V_sw_peak = max(Vin_list) + Vout + V_diode
print(f"V_sw_peak: {V_sw_peak}")

V_sw_peak: 30.55


# LTSpice Simulation
![](sepic_driver_circuit.png)

## 20 V input
![](sepic_fetdriver_20Vin.png)

### Soft Start (0->4 ms)
- Error-amplifier charges its soft-start capacitor.
- Internal soft-start delay is 4 ms.

### Steady State
- Reaches the right voltage approximately

## 5 V input
![](sepic_fetdriver_5Vin.png)

We're clearly dealing with subharmonic oscillations here.
Which leads us to 3 possible solutions
- Add a resistor for slope compensation
- Lower loop bandwidth
	- Change compensation circuit so controller won't run at this frequency
- Increase inductance to decrease current ramp.